In [1]:
import pandas as pd
import geopandas as gpd
import io
import json
import os

os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
metadata_df = pd.read_csv("output_metadata.csv")

In [2]:
from src.download import DataDownloaderUploader
from src.process import Processor
from src.document import DataDocumenter
from src.export import s3Exporter

In [3]:
#DataDownloaderUploader().download_and_upload_main()
#Processor().process_raw_datasets()
DataDocumenter().document_processed_data()
s3Exporter().export_all("export_test")

data/processed/administrative/boundaries_and_management/wind_river_reservation.geojson
data/processed/administrative/us_fish_and_wildlife/refuge_boundaries.geojson
data/processed/administrative/us_forest_service/priority_watersheds.geojson
data/processed/administrative/us_forest_service/special_interest_management_areas.geojson
data/processed/crucial_critical/crucial_range/antelope.geojson
data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson
data/processed/crucial_critical/crucial_range/elk.geojson
data/processed/crucial_critical/crucial_range/moose.geojson
data/processed/crucial_critical/crucial_range/mule_deer.geojson
data/processed/crucial_critical/crucial_range/rocky_mountain_goat.geojson
data/processed/crucial_critical/crucial_range/rocy_mountain_goat.geojson
data/processed/crucial_critical/crucial_range/whitetailed_deer.geojson
data/processed/crucial_critical/usfws_critical_habitat/canada_lynx.geojson
data/processed/crucial_critical/usfws_critical_habitat/critical_

In [5]:
gdf = gpd.read_file("wyoming.geojson")

In [2]:
gpd.read_file("wyoming.geojson").to_crs(32045).area


0    3.315330e+12
dtype: float64

In [3]:
gpd.read_file("wyoming.geojson").geometry.geom_type.value_counts()


Polygon    1
Name: count, dtype: int64

In [7]:
gdf.to_crs(32045).area.sum() / 60

55255496184.35588